# Statystyka w Analizie Danych

## Laboratorium 5 - algorytm normalizacja, selekcja cech.


### Opis
Celem laboratorium jest przeprowadzenie normalizacji i selekcji cech.


### Zbiór danych

Zbiór danych znajduje się w katalogu `dataset/*`. Jest to zmodyfikowany zbiór danych znajdujący się pod adresem: <https://archive.ics.uci.edu/ml/datasets/leaf>.

### Przesyłanie zadań

Wszystkie pliki należy spakować archiwizatorem **zip** i przesłać za pośrednictwem platformy WIKAMP. Poniżej oczekiwana zawartość archiwum:

```
+-- 📂 [IMIE.NAZWISKO].zip
    +-- 📜 Lab05.ipynb
    +-- 📂 dataset
        +-- 📜 dataset.npz
        +-- 📜 ReadMe.pdf
```

**Pamiętaj, wyniki powinny być czytelnie opisane oraz zaprezentowane graficznie (jeżeli jest taka możliwość).**

Przykład (na podstawie tablicy pomyłek):

**Źle** (nie wiadomo co jest poniżej zaprezentowane, kolumny ani wiersze nie są podpisane, nie wiadomo które z nich prezentują predykcje, a które właściwe etykiety):
```
array([[2, 0, 0],
       [0, 0, 1],
       [1, 0, 2]])
```

### Zadanie

Należy wykonać następujące czynności w celu realizacji niniejszego zadania:

#### Normalizacja
* Wczytaj dane.
* Znormalizuj dane.
* Przeprowadź eksperyment z zastosowaniem algorytmu kNN lub NM dla danych znormalizowanych oraz bez normalizacji.
    * W eksperymencie wybierz 5 klas oraz 10 cech.
* Przedstaw porównanie wyników klasyfikacji na danych znormalizowanych i bez normalizacji.
* Napisz wnioski.

**UWAGA: Wykorzystaj gotową implementację kNN [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)**

In [1]:
import numpy as np
from sklearn import neighbors

In [2]:
with open('./dataset/dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

In [3]:
def choose_classes(classes):
    for i in range(len(classes)):
        if i == 0:
            c_train = np.array(train[train[:,0] == classes[i]])
            c_test = np.array(test[test[:,0] == classes[i]])
        else:
            c_train = np.vstack((c_train,train[train[:,0] == classes[i]]))
            c_test = np.vstack((c_test,test[test[:,0] == classes[i]]))
    return c_train, c_test

classes = [2,3,5,8,10]
train, test = choose_classes(classes)

$$ x' = \frac{x- x_{min}}{x_{max}-x_{min}}$$

In [4]:
def normalize(arr):
    out = np.empty_like(arr)
    
    for i in range(arr.shape[1]):
        arr_min = np.min(arr[:,i])
        arr_max = np.max(arr[:,i])
        
        for p in range(arr.shape[0]):
            out[p,i] =  ((arr[p,i] - arr_min)/(arr_max-arr_min))*(1-0)+0
    
    return out

In [5]:
y_train = train[:,0]
x_train = train[:,2:]
y_test = train[:,0]
x_test = train[:,2:]

x_train.shape[1]

14

In [6]:
x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

In [7]:
norm_model = neighbors.KNeighborsClassifier(n_neighbors=3)
norm_model.fit(x_train_norm[:,1:11], y_train)
print(f'Skuteczność dla 5 klas i 10 cech po normalizacji:  {norm_model.score(x_test_norm[:,1:11],y_test):.2%}')

f_model = neighbors.KNeighborsClassifier(n_neighbors=3)
f_model.fit(x_train[:,1:11], y_train)
print(f'Skuteczność dla 5 klas i 10 cech bez normalizacji: {f_model.score(x_test[:,1:11],y_test):.2%}')

Skuteczność dla 5 klas i 10 cech po normalizacji:  100.00%
Skuteczność dla 5 klas i 10 cech bez normalizacji: 99.69%


#### Selekcja cech
Na tym samym podzbiorze danych (co w poprzednim zadaniu).
* Przeprowadź selekcję cech (wybierz {2, 5} cech) za pomocą metod poznanych na wykładzie (np. z zastosowaniem współczynnika Fishera) lub istniejących implementacji z biblioteki [scikit-learn](https://scikit-learn.org/stable/modules/feature_selection.html) (np. [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)).
* Przeprowadź klasyfikację na wybranych cechach.
* Porównaj wyniki klasyfikacji dla różnej liczby cech:
    * 10 cech bez normalizacji (wyniki z poprzedniego zadania),
    * 10 cech z normalizacją (wyniki z poprzedniego zadania),
    * 5 wybranych cech z bez normalizacji,
    * 5 wybranych cech z normalizacją,
    * 2 wybranych cech z bez normalizacji,
    * 2 wybranych cech z normalizacją.
* Opisz wyniki i napisz wnioski.


In [8]:
from sklearn.feature_selection import SelectKBest, chi2

In [14]:
# x_train_norm
# x_train
print(f'Skuteczność dla 5 klas i 10 cech bez normalizacji: {f_model.score(x_test[:,1:11],y_test):.2%}')
print(f'Skuteczność dla 5 klas i 10 cech po normalizacji:  {norm_model.score(x_test_norm[:,1:11],y_test):.2%}')
print()
x_train_norm_2 = SelectKBest(chi2, k=2).fit_transform(x_train_norm, y_train)
x_train_norm_5 = SelectKBest(chi2, k=5).fit_transform(x_train_norm, y_train)

x_train_2 = SelectKBest(k=2).fit_transform(x_train, y_train)
x_train_5 = SelectKBest(k=5).fit_transform(x_train, y_train)

#============================
model2 = SelectKBest(k=2)
x_train_2 =model2.fit_transform(x_train, y_train)
x_test_2 = model2.transform(x_test)

f_model_2 = neighbors.KNeighborsClassifier(n_neighbors=3)
f_model_2.fit(x_train_2, y_train)

print(f'Skuteczność dla 5 klas i 2 cech bez normalizacji: {f_model_2.score(x_test_2,y_test):.2%}')

#============================
model2norm = SelectKBest(k=2)
x_train_norm_2 =model2norm.fit_transform(x_train_norm, y_train)
x_test_norm_2 = model2norm.transform(x_test_norm)

norm_model_2 = neighbors.KNeighborsClassifier(n_neighbors=3)
norm_model_2.fit(x_train_norm_2, y_train)

print(f'Skuteczność dla 5 klas i 2 cech po normalizacji: {norm_model_2.score(x_test_norm_2,y_test):.2%}')
print()
#============================
model5 = SelectKBest(k=5)
x_train_5 =model5.fit_transform(x_train, y_train)
x_test_5 = model5.transform(x_test)

f_model_5 = neighbors.KNeighborsClassifier(n_neighbors=3)
f_model_5.fit(x_train_5, y_train)

print(f'Skuteczność dla 5 klas i 5 cech bez normalizacji: {f_model_5.score(x_test_5,y_test):.2%}')

#============================
model5norm = SelectKBest(k=5)
x_train_norm_5 =model5norm.fit_transform(x_train_norm, y_train)
x_test_norm_5 = model5norm.transform(x_test_norm)

norm_model_5 = neighbors.KNeighborsClassifier(n_neighbors=3)
norm_model_5.fit(x_train_norm_5, y_train)

print(f'Skuteczność dla 5 klas i 5 cech po normalizacji: {norm_model_5.score(x_test_norm_5,y_test):.2%}')


Skuteczność dla 5 klas i 10 cech bez normalizacji: 99.69%
Skuteczność dla 5 klas i 10 cech po normalizacji:  100.00%

Skuteczność dla 5 klas i 2 cech bez normalizacji: 87.77%
Skuteczność dla 5 klas i 2 cech po normalizacji: 88.40%

Skuteczność dla 5 klas i 5 cech bez normalizacji: 96.24%
Skuteczność dla 5 klas i 5 cech po normalizacji: 99.69%


- dla danych po normalizacji wyniki klasyfikacji są lepsze
- 5 najlepszych cech po normalizacji daje takie same wyniki jak 10 cech bez normalizacji